# Run a GWAS via regenie

In this notebook, we perform a genome-wide association study using [regenie](https://rgcgithub.github.io/regenie/) via [dsub](https://github.com/databiosphere/dsub).

# Setup 

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the All of Us Workbench.
    <ul>
        <li>Use compute type 'Standard VM' with sufficient CPU and RAM (e.g. start with 8 CPUs and 30 GB RAM, increase if needed).</li>
        <li>This notebook can take a while to run. Recommend that it is run in the background via <kbd>run_notebook_in_the_background</kbd>.</li>
    </ul>
</div>

In [ ]:
from datetime import datetime
import os
import pandas as pd
import time

## Setup plink2

https://www.cog-genomics.org/plink/2.0/

In [ ]:
%%bash

##### plink 2 install
PLINK_VERSION=2.3.Alpha
PLINK_ZIP_PATH=/tmp/plink-$PLINK_VERSION.zip
curl -L -o $PLINK_ZIP_PATH https://s3.amazonaws.com/plink2-assets/alpha2/plink2_linux_x86_64.zip
mkdir -p /tmp/plink2/
unzip -o $PLINK_ZIP_PATH -d /tmp/plink2/

In [ ]:
!/tmp/plink2/plink2 --version # --help

## Setup regenie

Note: regenie is already installed locally by default, but we are choosing to update to a more recent version.

In [ ]:
!regenie --version # --help

In [ ]:
%%bash

REGENIE_VERSION=v2.2.4
rm regenie.zip
curl -L -o regenie.zip "https://github.com/rgcgithub/regenie/releases/download/${REGENIE_VERSION}/regenie_${REGENIE_VERSION}.gz_x86_64_Linux.zip"
unzip -o regenie.zip

In [ ]:
!./regenie_v2.2.4.gz_x86_64_Linux --version # --help

## Define constants

The BGEN file created via `write_filtered_aou_bgen.ipynb`.

In [ ]:
REMOTE_BGEN = 'gs://fc-secure-.../PATH/TO/FILE.bgen'
REMOTE_BGEN_SAMPLE = 'gs://fc-secure-.../PATH/TO/FILE.sample'

LOCAL_BGEN = os.path.basename(REMOTE_BGEN)
LOCAL_BGEN_SAMPLE = os.path.basename(REMOTE_BGEN_SAMPLE)

These TSVs were created via notebook `AOU_UKB_phenotype_refined.ipynb`.

In [ ]:
REMOTE_PHENOTYPES = [
    'gs://fc-secure-.../PATH/TO/PHENOTYPES.csv',
]

LOCAL_PHENOTYPES = [os.path.basename(pheno) for pheno in REMOTE_PHENOTYPES]

In [ ]:
RESULT_BUCKET = os.getenv("WORKSPACE_BUCKET")
DATESTAMP = time.strftime('%Y%m%d')

# Outputs
OUTPUT_FILENAME_PREFIX = 'aou_alpha3_lipids'
REGENIE_PHENOTYPES = f'{OUTPUT_FILENAME_PREFIX}_phenotypes_and_covariates.tsv'
REGENIE_OUTPUTS = f'{os.getenv("WORKSPACE_BUCKET")}/data/aou/regenie/{DATESTAMP}/'

## Copy data locally for testing

In [ ]:
!gsutil cp -n {REMOTE_BGEN} {REMOTE_BGEN_SAMPLE} .

In [ ]:
for remote_pheno_file in REMOTE_PHENOTYPES:
    !gsutil cp -n {remote_pheno_file} .

# Reshape the phenotypes for regenie

**TODO(Margaret)** the code in this section was needed previously to reshape the phenotypes into the correct form for regenie. Either modify this code as needed, or remove it, if you update the R notebook to emit a phenotype file that can be used by regenie directly.

## Read in the four CSVs.

In [ ]:
raw_pheno_dfs = [pd.read_csv(local_pheno_file) for local_pheno_file in LOCAL_PHENOTYPES]

In [ ]:
[p.shape for p in raw_pheno_dfs]

In [ ]:
[p.columns for p in raw_pheno_dfs]

In [ ]:
raw_pheno_dfs[0].dtypes

## Join the four CSVs into a single CSV.

In [ ]:
index_col = ['eid', 'sampleid', 'sex', 'age', 'age2', 'pc1', 'pc2', 'pc3', 'pc4',
             'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10', 'statin']
pheno_dfs = [pd.read_csv(local_pheno_file, index_col=index_col) for local_pheno_file in LOCAL_PHENOTYPES]

In [ ]:
[p.shape for p in pheno_dfs]

In [ ]:
[p.columns for p in pheno_dfs]

In [ ]:
# Uncomment to see row level data.
#pheno_dfs[0].head()

In [ ]:
pheno_df = pd.concat(pheno_dfs, axis=1)

pheno_df.shape

## Check the resulting dataframe.

In [ ]:
pheno_df.dtypes

In [ ]:
pheno_df = pheno_df.reset_index()

In [ ]:
pheno_df.shape

In [ ]:
pheno_df.dtypes

In [ ]:
# Uncomment to see row level data.
#pheno_df.head()

In [ ]:
pheno_df = pheno_df.astype({'sampleid': 'int32'})

In [ ]:
# Uncomment to see row level data.
#pheno_df.head()

In [ ]:
pheno_df.groupby('sex')['sex'].count()

In [ ]:
len(pheno_df.sampleid)

In [ ]:
len(pheno_df.sampleid.unique())

In [ ]:
pheno_df[['HDLnorm', 'LDLnorm', 'TCnorm', 'TGnorm']].describe()

In [ ]:
[f'{col}: {pheno_df[col].isnull().sum()}' for col in sorted(pheno_df.columns)]

## Create the TSV for regenie.

### Add the  FID and IID columns.

In [ ]:
pheno_df['FID'] = pheno_df['sampleid']

In [ ]:
pheno_df['IID'] = pheno_df['FID']

In [ ]:
len(pheno_df.IID)

In [ ]:
len(pheno_df.IID.unique())

### Fill in NA covariates

In [ ]:
pheno_df['sex'] = pheno_df['sex'].fillna('other')

In [ ]:
[f'{col}: {pheno_df[col].isnull().sum()}' for col in sorted(pheno_df.columns)]

### Then write the TSV to disk so that regenie can read it.

In [ ]:
pheno_df.columns

In [ ]:
pheno_df[['FID', 'IID', 'sex', 'age', 'age2',
          'pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10',
          'HDLnorm', 'LDLnorm', 'TCnorm', 'TGnorm']].to_csv(
    REGENIE_PHENOTYPES,
    sep='\t',
    na_rep='NA',
    index=False
)

In [ ]:
# Uncomment to see row level data.
#!head {REGENIE_PHENOTYPES}

In [ ]:
!gsutil cp {REGENIE_PHENOTYPES} {REGENIE_OUTPUTS}

In [ ]:
!gsutil ls {REGENIE_OUTPUTS}

# Variant QC via PLINK

Use [plink2 to perform the variant QC](https://rgcgithub.github.io/regenie/recommendations/#exclusion-files) and obtain a subset of SNPs roughly equal to the number of samples.

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_BGEN} ref-first \
  --chr 1-22 \
  --geno 0.1 \
  --mind 0.1 \
  --mac 100 \
  --hwe 1e-15 \
  --write-snplist \
  --write-samples \
  --no-id-header \
  --out {OUTPUT_FILENAME_PREFIX}_plink

# This is too strict and removes too many samples.
#  --maf 0.01 \

In [ ]:
!ls -lth . | head

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_plink.id

In [ ]:
!tail {OUTPUT_FILENAME_PREFIX}_plink.id

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_plink.id

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_plink.snplist

In [ ]:
!tail {OUTPUT_FILENAME_PREFIX}_plink.snplist

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_plink.snplist

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}* {REGENIE_OUTPUTS}

# regenie

This work is based on https://github.com/briansha/Regenie_WDL/blob/master/regenie.wdl

See also:
* regenie documentation https://rgcgithub.github.io/regenie/options/#input
* dsub documentation https://github.com/DataBiosphere/dsub/blob/main/docs/input_output.md

## Step 1

From https://rgcgithub.github.io/regenie/overview/:
> In the first step a subset of genetic markers are used to fit a whole genome regression model that captures a good fraction of the phenotype variance attributable to genetic effects.

In [ ]:
# Parameters to add
# 8 core machine
# 11 GB ram
# 500 GB disk

!./regenie_v2.2.4.gz_x86_64_Linux \
    --step 1 \
    --bgen={LOCAL_BGEN} \
    --ref-first \
    --sample={LOCAL_BGEN_SAMPLE} \
    --phenoFile={REGENIE_PHENOTYPES} \
    --phenoColList=LDLnorm,HDLnorm,TCnorm,TGnorm \
    --covarFile={REGENIE_PHENOTYPES} \
    --catCovarList=sex \
    --covarColList=age,age2,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10 \
    --extract {OUTPUT_FILENAME_PREFIX}_plink.snplist \
    --bsize 1000 \
    --verbose \
    --out {OUTPUT_FILENAME_PREFIX}_regenie_part1


# Note that no samples were omitted by the QC step, so we are leaving out this file since regenie 
# complained that it did not match the samples in the BGEN file.
#     --keep {OUTPUT_FILENAME_PREFIX}.id \


In [ ]:
!ls -lth . | head

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}* {REGENIE_OUTPUTS}

In [ ]:
!gsutil ls {REGENIE_OUTPUTS}

## Step 2

From https://rgcgithub.github.io/regenie/overview/:
> In the second step, a larger set of genetic markers (e.g. imputed markers) are tested for association with the phenotype conditional upon the prediction from the regression model in Step 1, using a leave one chromosome out (LOCO) scheme, that avoids proximal contamination.

In [ ]:
!./regenie_v2.2.4.gz_x86_64_Linux \
    --step 2 \
    --bgen={LOCAL_BGEN} \
    --ref-first \
    --sample={LOCAL_BGEN_SAMPLE} \
    --phenoFile={REGENIE_PHENOTYPES} \
    --phenoColList=LDLnorm,HDLnorm,TCnorm,TGnorm \
    --covarFile={REGENIE_PHENOTYPES} \
    --catCovarList=sex \
    --covarColList=age,age2,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10 \
    --firth 0.01 \
    --approx \
    --pred {OUTPUT_FILENAME_PREFIX}_regenie_part1_pred.list \
    --bsize 400 \
    --out {OUTPUT_FILENAME_PREFIX}_regenie_part2

In [ ]:
!ls -lth {OUTPUT_FILENAME_PREFIX}*

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}* {REGENIE_OUTPUTS}

In [ ]:
!gsutil ls {REGENIE_OUTPUTS}

# Provenance 

In [ ]:
%%bash

date

In [ ]:
%%bash

pip3 freeze